In [1]:
from pathlib import Path

import pandas as pd

df = pd.read_pickle('/Users/daniildegtyarev/Downloads/BTCUSDT.pkl').rename(columns={
    'time_open' : 'open_time',
    'time_close' : 'close_time'
}).drop(columns=['interval', 'symbol_id'])
INDICATORS_CONFIG_PATH = Path(
    "/Users/daniildegtyarev/Projects/roehub.com/configs/prod/indicators.yaml"
)

In [7]:
import re
import time
from pathlib import Path

import numpy as np
import pandas as pd

try:
    from trading.contexts.backtest.adapters.outbound import YamlBacktestGridDefaultsProvider
    from trading.contexts.indicators.adapters.outbound.compute_numba import NumbaIndicatorCompute
    from trading.contexts.indicators.application.dto import CandleArrays, ComputeRequest
    from trading.contexts.indicators.domain.definitions import all_defs
    from trading.platform.config.indicators_compute_numba import (
        load_indicators_compute_numba_config,
    )
    from trading.shared_kernel.primitives import (
        MarketId,
        Symbol,
        Timeframe,
        TimeRange,
        UtcTimestamp,
    )
except ModuleNotFoundError as exc:
    raise RuntimeError(
        "Numba dependency is missing. Install project dependencies (for example: `uv sync`) "
        "and rerun this notebook."
    ) from exc

INDICATOR_IDS = ("ma.sma", "ma.ema", "trend.psar", "trend.adx", "trend.linreg_slope")
ROLLUP_TIMEFRAMES = ("1h",)
MARKET_ID = 1
SYMBOL = "BTCUSDT"

required_columns = {"open_time", "close_time", "open", "high", "low", "close", "volume"}
missing_columns = sorted(required_columns.difference(df.columns))
if missing_columns:
    raise ValueError(f"`df` is missing required columns: {missing_columns}")

for col in ("open_time", "close_time"):
    series = df[col]
    if not pd.api.types.is_datetime64_any_dtype(series):
        df[col] = pd.to_datetime(series, utc=True)
    elif getattr(series.dtype, "tz", None) is None:
        df[col] = series.dt.tz_localize("UTC")

df = df.sort_values("open_time").reset_index(drop=True)

if "INDICATORS_CONFIG_PATH" not in globals():
    INDICATORS_CONFIG_PATH = Path("configs/prod/indicators.yaml")

_SUPPORTED_ENGINE_TIMEFRAMES = {"1m", "5m", "15m", "30m" "1h", "4h", "1d"}
_WARNED_ENGINE_TIMEFRAMES: set[str] = set()


def _timeframe_to_rule_and_delta(timeframe: str) -> tuple[str, pd.Timedelta]:
    value = timeframe.strip().lower()
    matched = re.fullmatch(r"(\d+)\s*([mhd])", value)
    if matched is not None:
        amount = int(matched.group(1))
        unit = matched.group(2)
        if unit == "m":
            return (f"{amount}min", pd.Timedelta(minutes=amount))
        if unit == "h":
            return (f"{amount}h", pd.Timedelta(hours=amount))
        return (f"{amount}d", pd.Timedelta(days=amount))

    delta = pd.to_timedelta(value)
    if delta <= pd.Timedelta(0):
        raise ValueError(f"Invalid timeframe: {timeframe!r}")
    return (value, delta)


def rollup_ohlcv(frame: pd.DataFrame, *, timeframe: str) -> pd.DataFrame:
    rule, delta = _timeframe_to_rule_and_delta(timeframe)
    base = frame.sort_values("open_time").reset_index(drop=True)

    rolled = (
        base.set_index("open_time")
        .resample(rule, label="left", closed="left")
        .agg(
            open=("open", "first"),
            high=("high", "max"),
            low=("low", "min"),
            close=("close", "last"),
            volume=("volume", "sum"),
        )
        .dropna(subset=["open", "high", "low", "close"])
        .reset_index()
    )

    rolled["close_time"] = rolled["open_time"] + delta - pd.Timedelta(milliseconds=1)
    return rolled[["open_time", "close_time", "open", "high", "low", "close", "volume"]]


def _engine_timeframe_code(timeframe: str) -> str:
    value = timeframe.strip().lower()
    if value in _SUPPORTED_ENGINE_TIMEFRAMES:
        return value
    if value not in _WARNED_ENGINE_TIMEFRAMES:
        print(
            f"[warn] timeframe {value!r} is not supported by Timeframe primitive; "
            "using '1m' as CandleArrays metadata for compute."
        )
        _WARNED_ENGINE_TIMEFRAMES.add(value)
    return "1m"


def dataframe_to_candles(
    frame: pd.DataFrame,
    *,
    market_id: int,
    symbol: str,
    timeframe: str,
) -> CandleArrays:
    data = frame.sort_values("open_time").reset_index(drop=True)
    ts_open = (data["open_time"].astype("int64") // 1_000_000).to_numpy(dtype=np.int64, copy=True)

    start_dt = data["open_time"].iloc[0].to_pydatetime()
    end_dt = (data["close_time"].iloc[-1] + pd.Timedelta(milliseconds=1)).to_pydatetime()

    return CandleArrays(
        market_id=MarketId(market_id),
        symbol=Symbol(symbol),
        time_range=TimeRange(start=UtcTimestamp(start_dt), end=UtcTimestamp(end_dt)),
        timeframe=Timeframe(_engine_timeframe_code(timeframe)),
        ts_open=np.ascontiguousarray(ts_open, dtype=np.int64),
        open=np.ascontiguousarray(data["open"].to_numpy(), dtype=np.float32),
        high=np.ascontiguousarray(data["high"].to_numpy(), dtype=np.float32),
        low=np.ascontiguousarray(data["low"].to_numpy(), dtype=np.float32),
        close=np.ascontiguousarray(data["close"].to_numpy(), dtype=np.float32),
        volume=np.ascontiguousarray(data["volume"].to_numpy(), dtype=np.float32),
    )

print(f"Rows in input df: {len(df):,}")

Rows in input df: 879,241


In [8]:
provider = YamlBacktestGridDefaultsProvider.from_yaml(config_path=INDICATORS_CONFIG_PATH)

grids = {}
for indicator_id in INDICATOR_IDS:
    grid = provider.compute_defaults(indicator_id=indicator_id)
    if grid is None:
        raise ValueError(f"No defaults grid found for {indicator_id}")
    grids[indicator_id] = grid

numba_cfg = load_indicators_compute_numba_config(
    environ={
        "ROEHUB_ENV": "prod",
        "ROEHUB_INDICATORS_CONFIG": str(INDICATORS_CONFIG_PATH),
    }
)

engine = NumbaIndicatorCompute(defs=all_defs(), config=numba_cfg)
engine.warmup()

grid_rows = []
for indicator_id, grid in grids.items():
    estimate = engine.estimate(
        grid=grid,
        max_variants_guard=numba_cfg.max_variants_per_compute,
    )
    grid_rows.append(
        {
            "indicator_id": indicator_id,
            "variants": int(estimate.variants),
        }
    )

pd.DataFrame(grid_rows).sort_values("indicator_id").reset_index(drop=True)

,indicator_id,variants
0,ma.ema,1176
1,ma.sma,1176
2,trend.adx,5336
3,trend.linreg_slope,1176
4,trend.psar,369


In [9]:
rows = []

for timeframe in ROLLUP_TIMEFRAMES:
    rolled = rollup_ohlcv(df, timeframe=timeframe)
    candles = dataframe_to_candles(
        rolled,
        market_id=MARKET_ID,
        symbol=SYMBOL,
        timeframe=timeframe,
    )

    for indicator_id, grid in grids.items():
        started = time.perf_counter()
        status = "ok"
        error = ""
        variants = None

        try:
            tensor = engine.compute(
                ComputeRequest(
                    candles=candles,
                    grid=grid,
                    max_variants_guard=numba_cfg.max_variants_per_compute,
                )
            )
            variants = int(tensor.meta.variants)
        except Exception as exc:
            status = "error"
            error = f"{type(exc).__name__}: {exc}"

        elapsed_s = float(time.perf_counter() - started)
        rows.append(
            {
                "timeframe": timeframe,
                "bars": int(candles.ts_open.shape[0]),
                "indicator_id": indicator_id,
                "variants": variants,
                "elapsed_s": elapsed_s,
                "status": status,
                "error": error,
            }
        )

benchmark_df = pd.DataFrame(rows).sort_values(["timeframe", "indicator_id"]).reset_index(drop=True)
benchmark_df

[warn] timeframe '1h' is not supported by Timeframe primitive; using '1m' as CandleArrays metadata for compute.


,timeframe,bars,indicator_id,variants,elapsed_s,status,error
0,1h,73285,ma.ema,1176,0.660869,ok,
1,1h,73285,ma.sma,1176,1.031645,ok,
2,1h,73285,trend.adx,5336,3.010126,ok,
3,1h,73285,trend.linreg_slope,1176,2.887474,ok,
4,1h,73285,trend.psar,369,0.135242,ok,


In [10]:
ok_df = benchmark_df[benchmark_df["status"] == "ok"].copy()
if ok_df.empty:
    benchmark_df
else:
    ok_df.pivot(index="timeframe", columns="indicator_id", values="elapsed_s")